# SELECT A PSEDUO TESTING SET
I read and labelled a number of comments before generating the topics to have a small sample to test the results against

# CHECK AGAINST TESTING SET

In [11]:
import pandas as pd
import json
import os

# Set the path for the original data directory
path_original_data = r'C:\Users\Usuario\Documents\JupyterFolder\unimi_files\IR\files_txt'

# Load the JSON file into a dictionary
with open(os.path.join(path_original_data, 'testing_set.json'), 'r') as f:
    data_dict = json.load(f)

# Convert the keys of the dictionary from strings to integers
data_dict = {int(key): value for key, value in data_dict.items()}

# Set the path for the original data directory
path_original_data = r'C:\Users\Usuario\Documents\JupyterFolder\unimi_files\IR\files_csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(os.path.join(path_original_data, 'lpa_comment_data.csv'), low_memory=False)

# Create a list to store the selected rows
selected_rows = []

# Iterate over the keys in the dictionary and retrieve the corresponding rows from the database
for key in data_dict.keys():
    try:
        row_num = int(key)
        selected_rows.append(df.loc[row_num])
    except KeyError:
        print(f"Row {key} does not exist in the database.")

# Create a new DataFrame from the selected rows
selected_df = pd.DataFrame(selected_rows)

# Insert a new column 'row_id' at the first position with the original row number
selected_df.insert(0, 'row_id', selected_df.index)

# Print the selected rows with the 'row_id' column
selected_df.head(5)

,row_id,id,title,user,comment,rating,text_length,word_count,processed_comment,comment_tokenized,...,interaction_estimation,bash_the_leader_estimation,downtime_estimation,bookeeping_estimation,complex_or_complicated_relative_value,luck_relative_value,interaction_relative_value,bash_the_leader_relative_value,downtime_relative_value,bookeeping_relative_value
39652,39652,192135.0,Too Many Bones,Orphebus,Very original. Excesive production (and price)...,7.6,147,21,very original excesive production and price th...,"['very', 'original', 'excesive', 'production',...",...,0.147361,0.000000,0.000000,0.147620,3.064868,1.031528,1.031528,0.000000,0.000000,1.033339
32266,32266,230802.0,Azul,donnybrook23,This is a relatively quick game that is actual...,7.0,216,43,this is a relatively quick game that is actual...,"['this', 'is', 'a', 'relatively', 'quick', 'ga...",...,0.153463,0.000000,0.000000,0.000000,3.654245,0.000000,0.920781,0.000000,0.000000,0.000000
23835,23835,329551.0,Mosaic: A Story of Civilization,madchow,Played it last year at Gencon (prototype copy)...,9.0,514,101,played it last year at gencon prototype copy a...,"['played', 'it', 'last', 'year', 'at', 'gencon...",...,0.111896,0.079774,0.000000,0.070661,3.830608,1.566545,1.566545,1.116832,0.000000,0.989258
2165,2165,342942.0,Ark Nova,Eric Brosius,"This game was more enjoyable than I expected, ...",7.0,209,38,this game was more enjoyable than i expected p...,"['this', 'game', 'was', 'more', 'enjoyable', '...",...,0.411466,0.000000,0.000000,0.206669,0.000000,0.834094,1.645866,0.000000,0.000000,0.826674
12261,12261,205637.0,Arkham Horror: The Card Game,garysax,My review: https://therewillbe.games/articles...,8.0,608,74,my review httpstherewillbegamesarticlesboardg...,"['my', 'review', 'httpstherewillbegamesarticle...",...,0.156652,0.000000,0.258534,0.218963,4.274260,2.663077,2.663077,0.000000,4.395074,3.722369


### IMPORTANT VARIABLE

In [12]:
# PICK WHICH PERCENTAGES YOU WANT TO EXTRACT FROM THE DATABASE
n = 14  #14 for 'estimations' and 21 for 'relative values'

# Define the range of columns to extract
column_range = range(n, n+6)

#### SUPERVISED LABELS

In [13]:
topics = ['complex or complicated', 'luck', 'interaction', 'bash the leader', 'downtime', 'bookeeping']
count = 0

# Iterate over each row in the DataFrame
for index, row in selected_df.iterrows():
    all_res = 0
        
    comment = row['comment']
    row_number = row['row_id']
    confidance = int(row['dictionary_matches']) < 5

    # Extract values of columns
    values = [row[column] for column in column_range]

    # Convert values to strings
    values_as_strings = ["{:.2f}%".format(float(value)*100) for value in values]

    count = count + 1
    # Print the row number and comment as strings
    print(f"[{count}] ROW {row_number}:\n{comment}\n")

    if confidance: print('⚠️There is insufficient data for a precise analysis.\n')

    print('Label comparison: ')
    for label in data_dict[row_number]:
        status = '🔴'
        # Check the topic estimations
        for item1, item2 in zip(topics, values_as_strings):
            if item1 == label and str(item2) != '0.00%':
                status = '🟢'
                break        
        print(label.upper(), status)

    for item1, item2 in zip(topics, values_as_strings): 
        if item1 not in data_dict[row_number] and str(item2) != '0.00%': print(item1.upper(), '🟡')   

    # Print the topic estimations
    print('\nTopic estimation: ')
    for item1, item2 in zip(topics, values_as_strings): print('- ' + str(item1).upper() + ': ' + item2)

    print('\n--------------------------------------\n')            

[1] ROW 39652:
Very original. Excesive production (and price). Thematic, even narrative. I feel the replayability is limited, it will become repetitive over time.

Label comparison: 
COMPLEX OR COMPLICATED 🟢
LUCK 🟡
INTERACTION 🟡
BOOKEEPING 🟡

Topic estimation: 
- COMPLEX OR COMPLICATED: 43.78%
- LUCK: 14.74%
- INTERACTION: 14.74%
- BASH THE LEADER: 0.00%
- DOWNTIME: 0.00%
- BOOKEEPING: 14.76%

--------------------------------------

[2] ROW 32266:
This is a relatively quick game that is actually very easy to teach and is a game my whole family enjoy. Not a particularly complex game, but one that is good to dig out when you want something that is quick to play.

Label comparison: 
COMPLEX OR COMPLICATED 🟢
INTERACTION 🟡

Topic estimation: 
- COMPLEX OR COMPLICATED: 60.90%
- LUCK: 0.00%
- INTERACTION: 15.35%
- BASH THE LEADER: 0.00%
- DOWNTIME: 0.00%
- BOOKEEPING: 0.00%

--------------------------------------

[3] ROW 23835:
Played it last year at Gencon (prototype copy) and this year at 

#### ACCURACY

In [14]:
topics = ['complex or complicated', 'luck', 'interaction', 'bash the leader', 'downtime', 'bookeeping']
one_topic = "downtime"
topic_log = {}

# Declare variables to track efficiency analysis
all_res = 0
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0

for each in topics:
    topic_log[each] = []

    # Iterate over each row in the DataFrame
    for index, row in selected_df.iterrows():
        all_res = 0

        comment = row['comment']
        row_number = row['row_id']
        confidance = int(row['dictionary_matches']) < 5

        # Extract values of columns
        values = [row[column] for column in column_range]

        # Convert values to strings
        values_as_strings = ["{:.2f}%".format(float(value)*100) for value in values]

        for label in data_dict[row_number]:
            status = '🔴'
            # Check the topic estimations
            for item1, item2 in zip(topics, values_as_strings):
                if item1 == label and str(item2) != '0.00%':
                    status = '🟢'
                    break        
            if label == each: topic_log[each] = topic_log[each] + [status]

            if status == '🟢': true_pos = true_pos + 1
            else: false_neg = false_neg + 1  
            all_res = all_res + 1    

        for item1, item2 in zip(topics, values_as_strings): 
            if item1 not in data_dict[row_number] and str(item2) != '0.00%':
                if item1 == each: topic_log[each] = topic_log[each] + ['🟡']

                false_pos = false_pos + 1
                all_res = all_res + 1

        true_neg = true_neg + len(topics) - all_res  

In [15]:
# Calculate the accuracy
accuracy = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)
accuracy = "{:.2f}%".format(accuracy*100)
print(f"ACCURACY: {accuracy}\n")

temp = (true_pos) / (true_pos + false_neg)
temp = "{:.2f}%".format(temp*100)
print(f"POSSITIVE ACCURACY: {temp}")

temp = (true_neg) / (true_neg + false_pos)
temp = "{:.2f}%".format(temp*100)
print(f"NEGATIVE ACCURACY: {temp}")

from collections import Counter

for each in topics:
    # Print the count of each element
    print(f"\n{each.upper()}:")
    
    # Count the occurrences of each element
    count_dict = Counter(topic_log[each])
    
    print(count_dict)
    # for element, count in count_dict.items(): print(f"{element}: {count}")

ACCURACY: 62.92%

POSSITIVE ACCURACY: 69.12%
NEGATIVE ACCURACY: 60.47%

COMPLEX OR COMPLICATED:
Counter({'🟢': 21, '🔴': 5, '🟡': 4})

LUCK:
Counter({'🟡': 9, '🟢': 8})

INTERACTION:
Counter({'🟡': 12, '🔴': 9, '🟢': 5})

BASH THE LEADER:
Counter({'🟡': 17, '🟢': 2, '🔴': 2})

DOWNTIME:
Counter({'🟡': 16, '🔴': 3, '🟢': 3})

BOOKEEPING:
Counter({'🟡': 10, '🟢': 8, '🔴': 2})


### IMPORTANT VARIABLE

# CHECK RANDOM COMMENTS

In [5]:
# PICK WHICH PERCENTAGES YOU WANT TO EXTRACT FROM THE DATABASE
m = 13  #13 for 'estimations' and 20 for 'relative values'

# Define the range of columns to extract
other_column_range = range(m, m+6)

In [6]:
temp_df = df.sample(10)
temp_df.head(1)

,id,title,user,comment,rating,text_length,word_count,processed_comment,comment_tokenized,comment_key_words,...,interaction_estimation,bash_the_leader_estimation,downtime_estimation,bookeeping_estimation,complex_or_complicated_relative_value,luck_relative_value,interaction_relative_value,bash_the_leader_relative_value,downtime_relative_value,bookeeping_relative_value
38482,291457.0,Gloomhaven: Jaws of the Lion,Valiko76,Amazing. Way more easier to setup than Gloomha...,10.0,244,46,amazing way more easier to setup than gloomhav...,"['amazing', 'way', 'more', 'easier', 'to', 'se...","['amazing', 'way', 'easier', 'setup', 'gloomha...",...,0.07137,0.0,0.0,0.134613,1.884586,0.0,0.999176,0.0,0.0,1.884586


#### ESTIMATIONS

In [7]:
topics = ['complex or complicated', 'luck', 'interaction', 'bash the leader', 'downtime', 'bookeeping']

# Iterate over each row in the DataFrame
for index, row in temp_df.iterrows():
    comment = row['comment']
    row_number = index
    confidance = int(row['dictionary_matches']) < 5
    
    # Extract values of columns
    values = [row[column] for column in other_column_range]
    
    # Convert values to strings
    values_as_strings = ["{:.2f}%".format(float(value)*100) for value in values]
    
    # Print the row number and comment as strings
    print(f"ROW {row_number}:\n{comment}\n")
    
    if confidance: print('⚠️ There is insufficient data for a precise analysis.\n')
        
    # Print the topic estimations
    print('Topic estimation: ')
    for item1, item2 in zip(topics, values_as_strings): 
        #if item2 != '0.00%': 
        print('- ' + str(item1).upper() + ': ' + item2)
        
    print('\n--------------------------------------\n')  

ROW 38482:
Amazing. Way more easier to setup than Gloomhaven Core set. Nice story, and i love not having a thousand of tiles to search in the box. Now everything is in the manual. Ive played the fisrt three scenarios, love the pace of the XP progression !

Topic estimation: 
- COMPLEX OR COMPLICATED: 13.46%
- LUCK: 0.00%
- INTERACTION: 7.14%
- BASH THE LEADER: 0.00%
- DOWNTIME: 0.00%
- BOOKEEPING: 13.46%

--------------------------------------

ROW 27407:
I really enjoy this game solo so far. There are many different actions you can take, allowing you great variability in strategy and level of commitment.

Topic estimation: 
- COMPLEX OR COMPLICATED: 19.31%
- LUCK: 0.00%
- INTERACTION: 12.38%
- BASH THE LEADER: 0.00%
- DOWNTIME: 0.00%
- BOOKEEPING: 0.00%

--------------------------------------

ROW 3753:
Amazing game. Recommend ETSY bought inserts and player dashboards to protect those cubes. Don't listen to the Terraforming Mars comparisons. So not true and that's not a bad thing.

To

# ORDER THE DATABASE

### IMPORTANT VARIABLE

In [8]:
# TOPIC TO SEARCH
string = "luck"

# VARIABLE TO SEARCH
# value = '_relative_value'
value = '_estimation'

In [9]:
# Create a topic name by replacing spaces with underscores and appending the value
topic = string.replace(' ', '_') + value

# Sort the DataFrame based on the values in the specified topic column in descending order
df_sorted = df.sort_values(topic, ascending=False)

# Extract the 'comment' column of the first five rows
comments = df_sorted['comment'].head(5)

# Convert the comments to a list
comment_list = comments.tolist()

# Print each comment in the comment_list
for comment in comment_list:
    print(comment, '\n')

# Display the first five rows of the sorted DataFrame
df_sorted.head(5)


I enjoy the game but sometimes the randomness, especially the chaos tokens (drawn from the bag) can frustrate me. 

Decent bit of luck involved, but if you embrace that and roll with the punches it's a great time! 

Always fun to play. Note: you can have bad luck, and lose because of it, but when you are fine with it, you will have a very good time. 

You are with a monster.  Hit it. Draw token. Fail.  Draw token. Fail.  Draw token. Fail.  I want to leave - I can’t because I’m with a monster.  Hit it - draw token - succeed!  But it’s not dead yet.  Draw token. Fail. Draw token fail.  Draw token. Succeed. It’s dead!  Another monster has spawned.  Repeat ad infinitum.  Shite. 

I hate rolling dice but this game has made a believer out of me.  There is so much mitigation you can do with your dice roll that the game does not seem random even though you are rolling a ton of dice.  



,id,title,user,comment,rating,text_length,word_count,processed_comment,comment_tokenized,comment_key_words,...,interaction_estimation,bash_the_leader_estimation,downtime_estimation,bookeeping_estimation,complex_or_complicated_relative_value,luck_relative_value,interaction_relative_value,bash_the_leader_relative_value,downtime_relative_value,bookeeping_relative_value
14144,205637.0,Arkham Horror: The Card Game,ThatTrekBoy,"I enjoy the game but sometimes the randomness,...",8.0,113,19,i enjoy the game but sometimes the randomness ...,"['i', 'enjoy', 'the', 'game', 'but', 'sometime...","['enjoy', 'game', 'sometimes', 'randomness', '...",...,0.0,0.0,0.0,0.0,0.000000,3.226663,0.0,0.0,0.0,0.0
3886,342070.0,Thunder Road: Vendetta,NyceTries,"Decent bit of luck involved, but if you embrac...",9.0,97,19,decent bit of luck involved but if you embrace...,"['decent', 'bit', 'of', 'luck', 'involved', 'b...","['decent', 'bit', 'luck', 'involved', 'embrace...",...,0.0,0.0,0.0,0.0,0.000000,3.226660,0.0,0.0,0.0,0.0
30080,246900.0,Eclipse: Second Dawn for the Galaxy,DiaboloKlon,Always fun to play. Note: you can have bad luc...,9.0,135,29,always fun to play note you can have bad luck ...,"['always', 'fun', 'to', 'play', 'note', 'you',...","['always', 'fun', 'play', 'note', 'bad', 'luck...",...,0.0,0.0,0.0,0.0,0.000000,3.226630,0.0,0.0,0.0,0.0
11787,205637.0,Arkham Horror: The Card Game,dancj,You are with a monster. Hit it. Draw token. F...,1.4,328,59,you are with a monster hit it draw token fail...,"['you', 'are', 'with', 'a', 'monster', 'hit', ...","['monster', 'hit', 'draw', 'token', 'fail', 'd...",...,0.0,0.0,0.0,0.0,0.726047,3.940603,0.0,0.0,0.0,0.0
39451,192135.0,Too Many Bones,kentrisk,I hate rolling dice but this game has made a b...,9.0,203,42,i hate rolling dice but this game has made a b...,"['i', 'hate', 'rolling', 'dice', 'but', 'this'...","['hate', 'rolling', 'dice', 'game', 'made', 'b...",...,0.0,0.0,0.0,0.0,0.000000,3.126005,0.0,0.0,0.0,0.0
